In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import glob

#表示行数の増加
pd.set_option('display.max_rows', 500)

today = datetime.datetime.now()

y = today.year
m = today.month - 1

ym = str(y) + str('{0:02d}'.format(m))

#暫定でのroot dir
root = "/Users/miyamoto/Documents/Root/raw/weekly_agg"

#サイト単位で集計するショップを指定
shopUnit = \
["TOKYO OUTLET WEEK ONLINE", "ファッション通販 blanccloset", "ENVYM", "ECLIN", "公式通販Fig＆Viper", "DURAS OFFICIAL WEB STORE",\
 "EGOIST(エゴイスト)ファッション公式通販サイト", "COOMB official online store", "TONAL", \
 "TRUNC 88(トランク エイティーエイト)ブランド公式サイト", "Brown&Street ONLINE", "CUBE SUGAR",\
 "Cuore ONLINE SHOP", "Yeti イエティ", "BUTSU/YOKU", "Port Of Call Online", "willfully ONLINE SHOP","AURALEE",\
 "CAN ONLINE SHOP", "CAN OUTLET", "atomicboxx", "RABOKIGOSHI web store", "PALEMOBA ONLINESTORE", "ミツカル -bee able-",\
 "STRAWBERRY-FIELDS OUTLET", "SHIBUYA109"]

#ブランド単位で集計するショップを指定
brandUnit = \
["Arpege story", "Roomy's WEB STORE", "Ailand", "SANKO BAZAAR", "URBAN RESEARCH OUTLET","STRIPE CLUB", "KOE", "LEBECCA boutique"]

#KING
#上記は追い追い追加すること

#入庫集計対象のショップ（事業部指定）
receiveShop_convert = \
{'TOKYO OUTLET WEEK事業部':"TOKYO OUTLET WEEK ONLINE",\
 '英インターナショナル株式会社':"ファッション通販 blanccloset", \
 'ENVYM事業部':"ENVYM", \
 'ECLIN事業部':"ECLIN",\
 'FIG&VIPER':"公式通販Fig＆Viper",\
 'DURAS事業部':"DURAS OFFICIAL WEB STORE", 
 'COOMB事業部':"COOMB official online store", \
 'TRUNC 88事業部':"TRUNC 88(トランク エイティーエイト)ブランド公式サイト",
 "PALEMOBA事業部":"PALEMOBA ONLINESTORE",\
 '109事業部':"SHIBUYA109",\
 'willfully事業部':"willfully ONLINE SHOP",\
 'Brown&Street事業部':"Brown&Street ONLINE",\
 'AURALEE事業部':"AURALEE",\
 'ミツカル事業部':"ミツカル -bee able-",\
 'Yeti事業部':"Yeti イエティ",\
 'BUTSU/YOKU事業部':"BUTSU/YOKU",\
 'Cuore事業部':"Cuore ONLINE SHOP"\
 }

#入庫集計対象のショップ（ブランド単位）
receiveBrand_convert = \
{'STRIPE CLUB事業部':"STRIPE CLUB",\
 'Arpege story事業部':"Arpege story",\
 "Roomy's WEB STORE事業部":"Roomy's WEB STORE", \
 "Ailand事業部":"Ailand"}

#会員グループの変換用
regMember_convert = \
{"EGOIST":"EGOIST(エゴイスト)ファッション公式通販サイト", \
 "TOKYO OUTLET WEEK":"TOKYO OUTLET WEEK ONLINE",\
 "Brown&Street":"Brown&Street ONLINE",\
 'RABOKIGOSHI公式通販会員':"RABOKIGOSHI web store"}

In [2]:
def file_ditect(key_dir):#指定ディレクトリに存在するファイルパスの一覧取得
    path = root + "/" + key_dir + "/"
    
    files = []

    # パスで指定したファイルの一覧をリスト形式で取得
    files = glob.glob(path + '/*.csv')

    #print(files)

    for file in files:
        print(file)
    
    return files

In [3]:
#sell用の関数群

def import_sell(file):
    dataframe = pd.read_csv(file)
    dataframe = dataframe[~dataframe["ブランド名"].str.contains("ノベルティ")]
    
    dataframe.rename(columns={"日付":"年月日",'出荷金額-返品金額': '売上金額'}, inplace=True)
    
    return dataframe

def shop_sell(df):
    agg = df.query("ショップ名 in @shopUnit")
    
    if agg.empty:
        return
    
    agg = agg.groupby(["年月日", "ショップ名"]).agg({'出荷金額':'sum','返品金額':'sum','売上金額':'sum'})

    agg = agg.stack()
    
    return agg

def brand_sell(df):
    agg_brand = df.query("ショップ名 in @brandUnit")
    
    if agg_brand.empty:
        return

    agg_brand = agg_brand.groupby(["年月日", "ショップ名", "ブランド名"]).agg({'出荷金額':'sum','返品金額':'sum','売上金額':'sum'})

    agg_brand = agg_brand.stack().reset_index()

    return agg_brand


In [4]:
#sell処理
files_sell = file_ditect("sell")
sell_list_shop = []
sell_list_brand = []

for file in files_sell:
    sell = import_sell(file)
    df_shop_sell = shop_sell(sell)
    df_brand_sell = brand_sell(sell)
    sell_list_shop.append(df_shop_sell)
    sell_list_brand.append(df_brand_sell)

/Users/miyamoto/Documents/Root/raw/weekly_agg/sell/売上（出荷-顧客返品）_20210201091308_new.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/sell/売上（出荷-顧客返品）_20210201091843_TOW.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/sell/売上（出荷-顧客返品）_20210201091532_zero.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/sell/売上（出荷-顧客返品）_20210201091716_shibuya.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/sell/売上（出荷-顧客返品）_20210201091615_CAN.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/sell/売上（出荷-顧客返品）_20210201091807_AP.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/sell/売上（出荷-顧客返品）_20210201091424_stripe.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
sh_sell = pd.concat(sell_list_shop,axis = 0).reset_index()
sh_sell = sh_sell[sh_sell.columns[sh_sell.columns != 'index']]
sh_sell.rename(columns={"level_2":"属性", 0:"scalar"}, inplace =True)

br_sell = pd.concat(sell_list_brand,axis = 0).reset_index()
br_sell = br_sell[br_sell.columns[br_sell.columns != 'index']]
br_sell.rename(columns={"level_3":"属性", 0:"scalar"}, inplace =True)

In [6]:
sh_sell
#//sell処理

,年月日,ショップ名,属性,scalar
0,2020/12/01,ECLIN,出荷金額,266519.0
1,2020/12/01,ECLIN,返品金額,3500.0
2,2020/12/01,ECLIN,売上金額,263019.0
3,2020/12/01,ENVYM,出荷金額,87920.0
4,2020/12/01,ENVYM,返品金額,43500.0
...,...,...,...,...
3343,2021/01/30,CAN ONLINE SHOP,返品金額,42625.0
3344,2021/01/30,CAN ONLINE SHOP,売上金額,11028429.0
3345,2021/01/31,CAN ONLINE SHOP,出荷金額,13241033.0
3346,2021/01/31,CAN ONLINE SHOP,返品金額,0.0


In [7]:
#order用の関数群
def import_order(file):
    df = pd.read_csv(file)    
    df.rename(columns={'受注番号': '受注件数', '受注日': '年月日'}, inplace=True)
    df["販売ステータス"] = "受注金額/販売ステータス/" + df["販売ステータス"]
    
    return df

def shop_order(df):
    df = df.query("ショップ名 in @shopUnit")
    
    if df.empty:
        return
    
    agg = df.query("受注金額 != 0")
    agg = agg.groupby(["年月日", "ショップ名"]).agg({'受注件数':'nunique','受注数':'sum', '受注金額':'sum'})
    agg = agg.stack()
    
    agg_sellStatus = df.groupby(["年月日", "ショップ名", "販売ステータス"]).agg({'受注金額':'sum'})
    agg_sellStatus = agg_sellStatus.stack()
    
    df_agg = pd.concat([agg, agg_sellStatus], axis = 0)
    
    return df_agg
    
def brand_order(df):
    df = df.query("ショップ名 in @brandUnit")
    
    if df.empty:
        return
    
    agg = df.query("受注金額 != 0")
    agg = agg.groupby(["年月日", "ショップ名", "ブランド名"]).agg({'受注金額':'sum', '受注件数':'nunique','受注数':'sum'})
    agg = agg.stack()
    
    agg_site = df.query("受注金額 != 0")
    agg_site = agg_site.groupby(["年月日", "ショップ名"]).agg({'受注件数':'nunique','受注数':'sum'})
    agg_site = agg_site.stack()
    
    agg_sellStatus = df.groupby(["年月日", "ショップ名", "ブランド名", "販売ステータス"]).agg({'受注金額':'sum'})
    agg_sellStatus = agg_sellStatus.stack()
    
    df_agg = pd.concat([agg,  agg_sellStatus], axis = 0) 
    df_agg = df_agg.reset_index()
    df_agg.rename(columns={'level_3': '属性'}, inplace=True)
    
    df_agg_shop = agg_site
    df_agg_shop = df_agg_shop.reset_index()
    df_agg_shop.rename(columns={'level_2': '属性'}, inplace=True)

    agg_total = pd.concat([df_agg, df_agg_shop], axis = 0)
    
    return agg_total

In [8]:
#order処理
files_order = file_ditect("order")

order_list_shop = []
order_list_brand = []

for file in files_order:
    order = import_order(file)

    df_shop_order = shop_order(order)
    df_brand_order = brand_order(order)

    order_list_shop.append(df_shop_order)
    order_list_brand.append(df_brand_order)

/Users/miyamoto/Documents/Root/raw/weekly_agg/order/受注_20210201093520_new.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/order/受注_20210201094307_AP.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/order/受注_20210201094046_CAN.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/order/受注_20210201094229_shibuya.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/order/受注_20210201093954_zero.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/order/受注_20210201093635_stripe.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/order/受注_20210201094344_TOW.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (7,12,24,31,34,37,43) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (7,9,44) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memo

In [9]:
sh_order = pd.concat(order_list_shop,axis = 0).reset_index()
sh_order = sh_order[sh_order.columns[sh_order.columns != 'index']]
sh_order.rename(columns={"level_2":"属性", 0:"scalar"}, inplace =True)

br_order = pd.concat(order_list_brand,axis = 0).reset_index()
br_order = br_order[br_order.columns[br_order.columns != 'index']]
br_order.rename(columns={0:"scalar"}, inplace =True)
br_order = br_order[["年月日", "ショップ名", "ブランド名", "属性", "scalar"]]

In [10]:
sh_order.query("ショップ名 == 'CAN ONLINE SHOP' & 年月日 == '2020/10/01'")
#//order処理

,年月日,ショップ名,属性,scalar


In [11]:
#newProduct関数群　DLデータの調査が必要なため、明日の朝対応する
def import_newProduct(file):
    dataframe = pd.read_csv(file)
    dataframe = dataframe.astype({'年': str, '月': str, '日':str})
    dataframe["年月日"] = dataframe["年"]+"-"+dataframe["月"]+"-"+dataframe["日"]
    dataframe["年月日"] = pd.to_datetime(dataframe["年月日"])
    
    return dataframe

def shop_newProduct(df):
    df = df.query("ショップ名 in @shopUnit")
    
    if df.empty:
        return
    
    agg = df.groupby(["年月日", "ショップ名"]).agg({'品番数':'sum'})
    agg.rename(columns={'品番数':'新規アップ品番数'}, inplace=True)
    agg = agg.stack()
    
    return agg

def brand_newProduct(df):
    df = df.query("ショップ名 in @brandUnit")
    
    if df.empty:
        return
    
    agg = df.groupby(["年月日",  "ショップ名", "ブランド名"]).agg({'品番数':'sum'})
    agg.rename(columns={'品番数':'新規アップ品番数'}, inplace=True)
    agg = agg.stack()

    return agg

In [12]:
#newProduct処理
files_newProduct = file_ditect("newProduct")

newProduct_list_shop = []
newProduct_list_brand = []

for file in files_newProduct:
    newProduct = import_newProduct(file)

    df_shop_newProduct = shop_newProduct(newProduct)
    df_brand_newProduct = brand_newProduct(newProduct)

    newProduct_list_shop.append(df_shop_newProduct)
    newProduct_list_brand.append(df_brand_newProduct)

/Users/miyamoto/Documents/Root/raw/weekly_agg/newProduct/新規商品アップ数_20210201094551_zero.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/newProduct/新規商品アップ数_20210201094621_stripe.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/newProduct/新規商品アップ数_20210201094445_AP.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/newProduct/新規商品アップ数_20210201094650_new.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/newProduct/新規商品アップ数_20210201094413_TOW.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/newProduct/新規商品アップ数_20210201094519_CAN.csv


In [13]:
sh_newProduct = pd.concat(newProduct_list_shop,axis = 0).reset_index()
sh_newProduct = sh_newProduct[sh_newProduct.columns[sh_newProduct.columns != 'index']]
sh_newProduct.rename(columns={"level_2":"属性", 0:"scalar"}, inplace =True)

br_newProduct = pd.concat(newProduct_list_brand,axis = 0).reset_index()
br_newProduct = br_newProduct[br_newProduct.columns[br_newProduct.columns != 'index']]
br_newProduct.rename(columns={"level_3":"属性", 0:"scalar"}, inplace =True)

In [14]:
sh_newProduct
#//newProduct処理

,年月日,ショップ名,属性,scalar
0,2020-12-01,CUBE SUGAR,新規アップ品番数,1
1,2020-12-01,PALEMOBA ONLINESTORE,新規アップ品番数,14
2,2020-12-02,COOMB official online store,新規アップ品番数,2
3,2020-12-02,PALEMOBA ONLINESTORE,新規アップ品番数,37
4,2020-12-02,willfully ONLINE SHOP,新規アップ品番数,2
5,2020-12-03,EGOIST(エゴイスト)ファッション公式通販サイト,新規アップ品番数,3
6,2020-12-03,Port Of Call Online,新規アップ品番数,7
7,2020-12-03,STRAWBERRY-FIELDS OUTLET,新規アップ品番数,9
8,2020-12-03,TONAL,新規アップ品番数,2
9,2020-12-04,AURALEE,新規アップ品番数,21


In [15]:
#request関数群
def import_request(file):
    df = pd.read_csv(file)
    
    df["リクエスト金額"] = df["定価"]*df["登録数"]
    df["送信金額"] = df["定価"]*df["送信数"]
    
    df.rename(columns={'対象日': '年月日'}, inplace=True)
    
    #df = df.query("リクエストタイプ == '再入荷' ")
    
    return df

def shop_request(df):
    df = df.query("ショップ名 in @shopUnit")
    
    if df.empty:
        return
    
    df_reg = df.query("登録数 != 0")
    df_send = df.query("送信数 != 0")
    
    agg_reg = df_reg.groupby(["年月日", "ショップ名"]).agg({'登録数':'sum','リクエスト金額':'sum'})
    agg_reg.rename(columns={'登録数': 'リクエスト登録数'}, inplace=True)
    
    agg_send = df_send.groupby(["年月日", "ショップ名"]).agg({'送信数':'sum','送信金額':'sum'})
    agg_send.rename(columns={'送信数': 'リクエスト送信数'}, inplace=True)
    
    agg = pd.merge(agg_reg, agg_send, on=["年月日", "ショップ名"], how='left')
    
    agg = agg.stack()
    
    return agg

def brand_request(df):
    df = df.query("ショップ名 in @brandUnit")
    
    if df.empty:
        return
    
    df_reg = df.query("登録数 != 0")
    df_send = df.query("送信数 != 0")

    agg_reg = df_reg.groupby(["年月日", "ショップ名", "ブランド名"]).agg({'登録数':'sum','リクエスト金額':'sum'})
    agg_reg.rename(columns={'登録数': 'リクエスト登録数'}, inplace=True)

    agg_send = df_send.groupby(["年月日", "ショップ名", "ブランド名"]).agg({'送信数':'sum','送信金額':'sum'})
    agg_send.rename(columns={'送信数': 'リクエスト送信数'}, inplace=True)

    agg = pd.merge(agg_reg, agg_send, on=["年月日", "ショップ名", "ブランド名"], how='left')

    agg = agg.stack()

    return agg

In [16]:
#request処理
files_request = file_ditect("request")

request_list_shop = []
request_list_brand = []

for file in files_request:
    request = import_request(file)

    df_shop_request = shop_request(request)
    df_brand_request = brand_request(request)

    request_list_shop.append(df_shop_request)
    request_list_brand.append(df_brand_request)

/Users/miyamoto/Documents/Root/raw/weekly_agg/request/リクエストメール集計_20210201092244_stripe.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/request/リクエストメール集計_20210201092036_CAN.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/request/リクエストメール集計_20210201091952_AP.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/request/リクエストメール集計_20210201091914_TOW.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/request/リクエストメール集計_20210201092329_new.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/request/リクエストメール集計_20210201092113_zero.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (20,23,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (15,19,22,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [17]:
sh_request = pd.concat(request_list_shop,axis = 0).reset_index()
sh_request = sh_request[sh_request.columns[sh_request.columns != 'index']]
sh_request.rename(columns={"level_2":"属性", 0:"scalar"}, inplace =True)

br_request = pd.concat(request_list_brand,axis = 0).reset_index()
br_request = br_request[br_request.columns[br_request.columns != 'index']]
br_request.rename(columns={"level_3":"属性", 0:"scalar"}, inplace =True)

In [18]:
sh_request

,年月日,ショップ名,属性,scalar
0,2020/12/01,CAN ONLINE SHOP,リクエスト登録数,1251.0
1,2020/12/01,CAN ONLINE SHOP,リクエスト金額,8765440.0
2,2020/12/01,CAN ONLINE SHOP,リクエスト送信数,87.0
3,2020/12/01,CAN ONLINE SHOP,送信金額,497900.0
4,2020/12/02,CAN ONLINE SHOP,リクエスト登録数,1661.0
...,...,...,...,...
3613,2021/01/31,Yeti イエティ,リクエスト金額,160900.0
3614,2021/01/31,willfully ONLINE SHOP,リクエスト登録数,164.0
3615,2021/01/31,willfully ONLINE SHOP,リクエスト金額,2143100.0
3616,2021/01/31,willfully ONLINE SHOP,リクエスト送信数,46.0


In [19]:
#receive関数群
def import_receive(file):
    df = pd.read_csv(file)
    
    df.rename(columns={'入庫実施日':'年月日','入庫実績':'入庫金額'}, inplace=True)
    
    df = df[~df["ブランド名"].str.contains("ノベルティ")]
    
    return df

def shop_receive(df):
    df = df.replace(receiveShop_convert)
    df = df.query("事業部 in @shopUnit")
    
    if df.empty:
        return
    
    agg = df.groupby(["年月日", "事業部"]).agg({'入庫金額':'sum'})
    agg = agg.stack()
    
    return agg

def brand_receive(df):
    df = df.replace(receiveBrand_convert)
    df = df.query("事業部 in @brandUnit")
    
    if df.empty:
        return
    
    if df.事業部.unique()[0] == "STRIPE CLUB":
        df["入庫金額"] = df.入庫実績数 * df.定価
    
    agg = df.groupby(["年月日", "事業部", "ブランド名"]).agg({'入庫金額':'sum'})
    
    agg = agg.stack()

    return agg

In [20]:
#receive処理
files_receive = file_ditect("receive")

receive_list_shop = []
receive_list_brand = []

for file in files_receive:
    receive = import_receive(file)

    df_shop_receive = shop_receive(receive)
    df_brand_receive = brand_receive(receive)

    receive_list_shop.append(df_shop_receive)
    receive_list_brand.append(df_brand_receive)

/Users/miyamoto/Documents/Root/raw/weekly_agg/receive/入庫実績_20210201093224_AP.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/receive/入庫実績_20210201093318_zero.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/receive/入庫実績_20210201093158_TOW.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/receive/入庫実績_20210201093442_new.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/receive/入庫実績_20210201093248_shibuya.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/receive/入庫実績_20210201093353_stripe.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (15,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [21]:
sh_receive = pd.concat(receive_list_shop,axis = 0).reset_index()
sh_receive = sh_receive[sh_receive.columns[sh_receive.columns != 'index']]
sh_receive.rename(columns={"level_2":"属性", 0:"scalar","事業部":"ショップ名"}, inplace =True)

br_receive = pd.concat(receive_list_brand,axis = 0).reset_index()
br_receive = br_receive[br_receive.columns[br_receive.columns != 'index']]
br_receive.rename(columns={"level_3":"属性", 0:"scalar","事業部":"ショップ名"}, inplace =True)

In [22]:
sh_receive

,年月日,ショップ名,属性,scalar
0,2020/12/01,BUTSU/YOKU,入庫金額,22700
1,2020/12/01,Brown&Street ONLINE,入庫金額,1098000
2,2020/12/01,PALEMOBA ONLINESTORE,入庫金額,572500
3,2020/12/01,willfully ONLINE SHOP,入庫金額,2632300
4,2020/12/01,ミツカル -bee able-,入庫金額,110700
5,2020/12/02,AURALEE,入庫金額,9697000
6,2020/12/02,Brown&Street ONLINE,入庫金額,150100
7,2020/12/02,COOMB official online store,入庫金額,143400
8,2020/12/02,PALEMOBA ONLINESTORE,入庫金額,1715500
9,2020/12/02,TRUNC 88(トランク エイティーエイト)ブランド公式サイト,入庫金額,191000


In [23]:
#regMember関数群
def import_regMember(file):
    df = pd.read_csv(file)
    
    df.rename(columns={'申込日':'年月日'}, inplace=True)
    
    if "CAN" in file:
        df.rename(columns={'登録サイト':'会員グループ'}, inplace=True)
    if "stripe" in file:
        df.rename(columns={'登録サイト':'会員グループ'}, inplace=True)
    
    
    df.rename(columns={'会員グループ':'ショップ名'}, inplace=True)
    
    df = df.replace(regMember_convert)
    
    return df

def shop_regMember(df):
    df = df.query("ショップ名 in @shopUnit")

    if df.empty:
        return
    
    agg = df.groupby(["年月日", "ショップ名", "申込種別"]).agg({'顧客コード':'nunique'})
    agg.rename(columns={'顧客コード':'申込件数'}, inplace=True)
    
    agg = agg.stack().reset_index()
    
    agg["属性"] = agg["level_3"] + "/" + agg["申込種別"]
    
    return agg

def brand_regMember(df):
    df = df.query("ショップ名 in @brandUnit")

    if df.empty:
        return
    
    agg = df.groupby(["年月日", "ショップ名", "申込種別"]).agg({'顧客コード':'nunique'})
    agg.rename(columns={'顧客コード':'申込件数'}, inplace=True)
    
    agg = agg.stack().reset_index()
    
    agg["属性"] = agg["level_3"] + "/" + agg["申込種別"]
    
    return agg

In [24]:
#regMember処理
files_regMember = file_ditect("regMember")

regMember_list_shop = []
regMember_list_brand = []

for file in files_regMember:
    regMember = import_regMember(file)
    
    df_shop_regMember = shop_regMember(regMember)
    df_brand_regMember = brand_regMember(regMember)

    regMember_list_shop.append(df_shop_regMember)
    regMember_list_brand.append(df_brand_regMember)

/Users/miyamoto/Documents/Root/raw/weekly_agg/regMember/正会員数獲得データ_日付絞込_20210201092854_shibuya.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/regMember/正会員数獲得データ_日付絞込_20210201092430_new.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/regMember/正会員数獲得データ_日付絞込_20210201092524_stripe.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/regMember/正会員数獲得データ_日付絞込_20210201093131_TOW.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/regMember/正会員数獲得データ_日付絞込_20210201092625_zero.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/regMember/正会員数獲得データ_日付絞込_20210201093048_AP.csv
/Users/miyamoto/Documents/Root/raw/weekly_agg/regMember/正会員数獲得データ_日付絞込_20210201092712_CAN.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [25]:
sh_regMember = pd.concat(regMember_list_shop,axis = 0).reset_index()
sh_regMember = sh_regMember[sh_regMember.columns[sh_regMember.columns != 'index']]
sh_regMember.rename(columns={0:"scalar"}, inplace =True)
sh_regMember = sh_regMember[["年月日", "ショップ名", "属性", "scalar"]]

br_regMember = pd.concat(regMember_list_brand,axis = 0).reset_index()
br_regMember = br_regMember[br_regMember.columns[br_regMember.columns != 'index']]
br_regMember.rename(columns={0:"scalar"}, inplace =True)
br_regMember = br_regMember[["年月日", "ショップ名", "属性", "scalar"]]

In [26]:
br_regMember

,年月日,ショップ名,属性,scalar
0,2020/12/01,Ailand,申込件数/会員登録,57
1,2020/12/01,Ailand,申込件数/会員解除,12
2,2020/12/01,Roomy's WEB STORE,申込件数/会員登録,65
3,2020/12/01,Roomy's WEB STORE,申込件数/会員解除,11
4,2020/12/02,Ailand,申込件数/会員登録,37
...,...,...,...,...
910,2021/01/29,Arpege story,申込件数/会員解除,17
911,2021/01/30,Arpege story,申込件数/会員登録,89
912,2021/01/30,Arpege story,申込件数/会員解除,8
913,2021/01/31,Arpege story,申込件数/会員登録,113


In [27]:
sh_total = pd.concat([sh_sell, sh_order, sh_newProduct, sh_request, sh_receive, sh_regMember], axis = 0) #sh_receive,
br_total = pd.concat([br_sell, br_order, br_newProduct, br_request, br_receive, br_regMember], axis = 0)

sh_total = sh_total[["年月日", "ショップ名","属性", "scalar"]].reset_index()
sh_total = sh_total[sh_total.columns[sh_total.columns != 'index']]

br_total = br_total[["年月日", "ショップ名", "ブランド名", "属性", "scalar"]].reset_index()
br_total = br_total[br_total.columns[br_total.columns != 'index']]

all_total = pd.concat([sh_total, br_total], axis = 0)
all_total = all_total[["年月日", "ショップ名", "ブランド名", "属性", "scalar"]]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [28]:
all_total

,年月日,ショップ名,ブランド名,属性,scalar
0,2020/12/01,ECLIN,NaN,出荷金額,266519.0
1,2020/12/01,ECLIN,NaN,返品金額,3500.0
2,2020/12/01,ECLIN,NaN,売上金額,263019.0
3,2020/12/01,ENVYM,NaN,出荷金額,87920.0
4,2020/12/01,ENVYM,NaN,返品金額,43500.0
...,...,...,...,...,...
41549,2021/01/29,Arpege story,NaN,申込件数/会員解除,17.0
41550,2021/01/30,Arpege story,NaN,申込件数/会員登録,89.0
41551,2021/01/30,Arpege story,NaN,申込件数/会員解除,8.0
41552,2021/01/31,Arpege story,NaN,申込件数/会員登録,113.0


In [29]:
all_total.to_csv('weekly.csv', encoding='utf_8_sig')